In [1]:
from transformers import ElectraTokenizer, ElectraModel
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.optim import AdamW
import transformers

In [2]:
import pandas as pd
import numpy as np

In [3]:
df =  pd.read_csv("data.csv")

In [4]:
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

In [5]:
RANDOM_SEED=1

In [15]:
model =  ElectraModel.from_pretrained('google/electra-small-discriminator')

In [29]:
def get_vector_representation(article):
    inputs = tokenizer(article["title"]+article["content"], return_tensors="pt")
    return torch.mean(model(**(inputs))[0][0])

In [30]:
df["vector_representation"]=df.apply(lambda x: get_vector_representation(x),axis=1)

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 952576 bytes. Buy new RAM!


In [7]:
class FakeNewsDataset(Dataset):
  def __init__(self, tokenizer, titles,content, labels,vectors):
    self.titles = titles
    self.content = content
    self.labels = labels
    self.vectors=vectors
    self.tokenizer = tokenizer
    self.max_len =160

  def __len__(self):
    return len(self.titles)


  def __getitem__(self, id):
    title = str(self.titles[id])
    content = str(self.content[id])
    label=self.labels[id]
    vector=self.vectors[id]

    return {
      'text': title+content,
      'vector': vector,
      'labels':label
    }

In [8]:
df_train, df_test = train_test_split(
  df,
  test_size=0.1, random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5, random_state=RANDOM_SEED
)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
def get_loader(df, tokenizer, batch_size):
  ds = FakeNewsDataset(
    titles=df.title.to_numpy(),
    content=df.content.to_numpy(),
    labels=df.label.to_numpy(),
    vectors=df.vector_representation.to_numpy()
  )
  return DataLoader(
    ds,
    batch_size=batch_size
  )

In [11]:
BATCH_SIZE = 16
train_data_loader = get_loader(df_train, tokenizer, BATCH_SIZE)
val_data_loader = get_loader(df_val, tokenizer, BATCH_SIZE)
test_data_loader = get_loader(df_test, tokenizer, BATCH_SIZE)

In [19]:
class SentimentClassifier(nn.Module):
  def __init__(self, class_size):
    super(SentimentClassifier, self).__init__()
    self.out = nn.Linear(model.config.hidden_size, class_size)

  def forward(self, vector):

    output = self.drop(vector)

        
    return self.out(output)

In [20]:
classifier=SentimentClassifier(2)

In [21]:
EPOCHS=3

optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = transformers.get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [22]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    print("1")
    _,preds = torch.max(d["vector"], dim=1)
    loss = loss_fn(d["vector"], labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [23]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      vector = d["vector"].to(device)
      labels = d["labels"].to(device)

      _,preds = torch.max(vector, dim=1)
      loss = loss_fn(vector, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [24]:
from collections import defaultdict

In [25]:
%%time
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    classifier,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    classifier,
    val_data_loader,
    loss_fn,
    device,
    len(df_val)
  )
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/3
----------
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


NameError: name 'input_ids' is not defined